# AI!



This snippet of code uses nbmake's NotebookRun class to test `res/x.ipynb` from a Python application. It starts by setting the `path` to the notebook, and the `timeout` for the notebook execution. Then a `NotebookRun` instance is created using the `filename` and `default_timeout` arguments. The notebook is then executed, and the result is checked for errors. If an error is found, it is printed out, otherwise a success message is printed.

In [ ]:

# Example Usage
if __name__ == "__main__":
    # Path of the notebook to run
    path = Path("res/x.ipynb")

    # Timeout for the notebook execution
    timeout = 10

    # Create a NotebookRun instance
    run = NotebookRun(filename=path, default_timeout=timeout)

    # Execute the notebook
    res = run.execute()

    # Check for errors
    if res.error is not None:
        print(f"Error: {res.error.summary}")
        print(f"Traceback:\n{res.error.trace}")
    else:
        print("Notebook execution successful")